In [1]:
#sympy for symbolic equations, latex2sympy for converting latex equations 

import sympy as sp
from latex2sympy2 import latex2sympy

### Core Parser Function

In [2]:
def parse_latex_to_sympy(latex_str: str):
    """
    Convert LaTeX math expression to SymPy object.

    Returns:
        sympy expression
    Raises:
        ValueError if parsing fails
    """
    try:
        expr = latex2sympy(latex_str)
        return sp.simplify(expr)
    except Exception as e:
        raise ValueError(f"Invalid LaTeX input: {e}")


In [6]:
examples = [
    r"x^2 + 2x + 1",
    r"\int_0^1 x^2 dx",
    r"\frac{d}{dx} x^3",
    r"\begin{bmatrix}1 & 2 \\ 3 & 4\end{bmatrix}"
]

for ex in examples:
    print("Input:", ex)
    print("Parsed:", parse_latex_to_sympy(ex))
    print()


Input: x^2 + 2x + 1
Parsed: x**2 + 2*x + 1

Input: \int_0^1 x^2 dx
Parsed: Integral(x**2, (x, 0, 1))

Input: \frac{d}{dx} x^3
Parsed: 3*x**2

Input: \begin{bmatrix}1 & 2 \\ 3 & 4\end{bmatrix}
Parsed: Matrix([[1, 2], [3, 4]])



In [7]:
expr = parse_latex_to_sympy(r"\int_0^1 x^2 dx")
type(expr)

sympy.integrals.integrals.Integral